In [1]:
sukinda = '/Users/angus/work/sukinda/lab_work/cations/raw_data/2025_01_09_Angus_River_waters.csv'
deccan_soils = '/Users/angus/work/sukinda/lab_work/cations/raw_data/2025_01_13_ANGUS_RIVER_WATERS.csv'

file_paths = [sukinda, deccan_soils]
file_names = ['data/sukinda.nc', 'data/deccan_soils.nc']

for file_path, file_name in zip(file_paths, file_names):
    %run process.py $file_path $file_name 

Data processed and saved to data/sukinda.nc
Data processed and saved to data/deccan_soils.nc


/Users/angus/work/icp_oes/import_icpoes.py:188: UserWarning: rename 'sample_name' to 'sample_name' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)
/Users/angus/work/icp_oes/import_icpoes.py:188: UserWarning: rename 'species' to 'species' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)
/Users/angus/work/icp_oes/import_icpoes.py:188: UserWarning: rename 'sample_name' to 'sample_name' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)
/Users/angus/work/icp_oes/import_icpoes.py:188: UserWarning: rename 'species' to 'species' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_m

In [3]:
# Split the Deccan and soil datasets.
import xarray as xr
ds = xr.open_dataset('data/deccan_soils.nc')

from utils import match_sample_name

labels_to_split = match_sample_name(ds, 'D', threshold=0.4).coords['sample_name'].values.tolist()
labels_to_split.extend(['Sukinda Filter Blank'])
print(labels_to_split)  
# Define the list of sample names that should appear in both datasets.
common_samples = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
check_stds = match_sample_name(ds, ['SPS-SW2 10%', 'SLRS-6', 'Cam-Tap-Water']).coords['sample_name'].values.tolist()
common_samples.extend(check_stds)    # These will be included in both ds_selected and ds_remaining
print(common_samples)
# Get all sample names from the dataset in their original order.
all_labels = ds.coords['sample_name'].values.tolist()

# Build the list for the first dataset:
# Include samples if they are either in labels_to_split or in common_samples.
selected_labels = [name for name in all_labels if name in labels_to_split or name in common_samples]

# Build the list for the second dataset:
# Include samples if they are not in labels_to_split, unless they are in common_samples.
# This ensures that any sample in common_samples appears in both datasets.
remaining_labels = [name for name in all_labels if (name not in labels_to_split) or (name in common_samples)]

# Select the datasets based on the computed labels.
ds_selected = ds.sel(sample_name=selected_labels)
ds_remaining = ds.sel(sample_name=remaining_labels)

['D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D1', 'D20', 'D21', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D', 'Sukinda Filter Blank']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Cam-Tap-Water_1', 'Cam-Tap-Water_2', 'Cam-Tap-Water_3', 'Cam-Tap-Water_4', 'Cam-Tap-Water_5', 'SLRS-6_1', 'SLRS-6_2', 'SLRS-6_3', 'SLRS-6_4', 'SLRS-6_5', 'SPS-SW2 10%_1', 'SPS-SW2 10%_2', 'SPS-SW2 10%_3', 'SPS-SW2 10%_4', 'SPS-SW2 10%_5']
Selected Dataset:
<xarray.Dataset>
Dimensions:                      (sample_name: 45, species: 39)
Coordinates:
  * sample_name                  (sample_name) object 'A' ... 'Sukinda Filter...
  * species                      (species) object 'Al 396.152 Ax' ... 'Sr 421...
Data variables: (12/29)
    Flags                        (sample_name, species) object ...
    Unadjusted_Data              (sample_name, species) float64 ...
    Concentration                (sample_name, species) float64 ...
    Intensity                    (sample_name, spe

In [4]:
# Reorder sample coordinates
from utils import reorder_sample_coord

ds_deccan = reorder_sample_coord(ds_selected)
ds_soils = reorder_sample_coord(ds_remaining)
ds_deccan.to_netcdf('data/deccan.nc')
ds_soils.to_netcdf('data/soils.nc')

ds_sukinda = xr.open_dataset('data/sukinda.nc').load()
ds_sukinda = reorder_sample_coord(ds_sukinda)
ds_sukinda.to_netcdf('data/sukinda.nc')